In [14]:
import re
import requests
import os
from datetime import datetime

from bs4 import BeautifulSoup
from bs4.element import Tag

import pandas as pd
from openai import OpenAI
#Call the openai api directly
# Initialize OpenAI client
open_ai_key = "sk-YApzULCHGCaBlngoQKKjQqI9eMTjAlZsyBRIae3dMM8WvXXX"
client = OpenAI(
    api_key=open_ai_key,
    base_url="https://api.chatanywhere.tech/v1"
)

def get_scraper_task(instruction,model="gpt-4o"):
    """
    Generates a web scraping task based on the provided instruction.

    Args:
        instruction (str): The instruction containing the task details and URL.

    Returns:
        str: The response content from the model or an error message if the API call fails.
    """
    try:
        # Call the OpenAI API with the provided instruction
        completion = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a web scraper. Follow the prompt in instruction to do task with the URL in the instruction.(note：You can use any way to get the html of a web page)"},
                {"role": "user", "content": instruction}
            ]
        )
        
        # Return the content of the model's response
        return completion.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"


In [16]:
def fetch_webpage_and_query(prompt, url, model="gpt-4o"):
    """
    Fetches the HTML content of a given URL and queries a model with the prompt.

    Args:
        prompt (str): The question or instruction for the model.
        url (str): The webpage URL to scrape.
        model (str): The model to be used for querying (default is "gpt-4o").

    Returns:
        dict: The response from the model.
    """
    try:
        # Fetch HTML content from the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for HTTP issues
        html_content = response.text

        # Construct the instruction dictionary
        instruction = {
            "prompt": prompt,
            "URL": url,
            "html": html_content,
        }

        # Convert the instruction to JSON string
        instruction_str = json.dumps(instruction)

        # Call the scraping task function with the model
        model_response = get_scraper_task(instruction_str, model)

        return model_response

    except requests.exceptions.RequestException as e:
        return {"error": f"Failed to fetch the webpage: {e}"}
    except Exception as e:
        return {"error": f"An error occurred: {e}"}


# dog page

In [11]:
from scrapegraphai.graphs import CustomGraph
import nest_asyncio
nest_asyncio.apply()


open_ai_key = "sk-YApzULCHGCaBlngoQKKjQqI9eMTjAlZsyBRIae3dMM8WvXXX"
graph_config = {
    "llm": {
        "api_key": open_ai_key,
        "model": "openai/gpt-4o",
        "base_url": "https://api.chatanywhere.tech/v1"
    },
    
    "depth": 1,  
    "only_inside_links": True,
    "verbose": True,
    "headless": True
    
    
}
graph = CustomGraph(
    prompt="List me all the dog breeds",
    source="https://www.yappy.com/uk/dogs/breeds",
    config=graph_config
)

result = graph.run()
print(result)

--- Executing FetchLevelK Node ---
Fetching HTML content from: https://www.yappy.com/uk/dogs/breeds
Attempt 1 failed: 
Fetched content for https://www.yappy.com/uk/dogs/breeds at depth 1
Extracted 535 links.
--- Executing ParseNode Node ---
--- Executing RAG Node ---
Creating collection 'vectorial_collection'...
Upserted 3 documents into the vector database.
--- Executing GenerateAnswer Node ---
Processing chunks: 100%|███████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1065.45it/s]


{'DogBreeds': ['Affenpinscher', 'Afghan Hound', 'Airedale Terrier', 'Akita', 'Alaskan Klee Kai', 'Alaskan Malamute', 'American Bulldog', 'American Cocker Spaniel', 'American Eskimo Dog', 'American Hairless Terrier', 'American Leopard Hound', 'American Pit Bull Terrier', 'American Staffordshire Terrier', 'American Water Spaniel', 'Anatolian Shepherd Dog', 'Aussiedoodle', 'Australian Cattle Dog', 'Australian Kelpie', 'Australian Labradoodle', 'Australian Shepherd', 'Australian Silky Terrier', 'Australian Terrier', 'Australian Working Kelpie', 'Azawakh', 'Basenji', 'Bassador', 'Basset Bleu De Gascogne', 'Basset Fauve De Bretagne', 'Basset Hound', 'Bassugg', 'Bavarian Mountain Hound', 'Beagador', 'Beagle', 'Beaglier', 'Bearded Collie', 'Beauceron', 'Bedlington Terrier', 'Bedlington Whippet', 'Belgian Groenendael', 'Belgian Laekenois', 'Belgian Malinois', 'Belgian Shepherd', 'Belgian Tervuren', 'Bergamasco', 'Bernedoodle', 'Bernese Mountain Dog', 'Bichon Frise', 'Bichon Yorkie', 'Bich-poo',

In [18]:
prompt = "List me all the dog breeds"
url = "https://www.yappy.com/uk/dogs/breeds"
model = "gpt-4o"
response = fetch_webpage_and_query(prompt, url, model)
print(response)

Here is a list of dog breeds from the provided webpage:

1. Affenpinscher
2. Afghan Hound
3. Airedale Terrier
4. Akita
5. Alaskan Klee Kai
6. Alaskan Malamute
7. American Bulldog
8. American Cocker Spaniel
9. American Eskimo Dog
10. American Hairless Terrier
11. American Leopard Hound
12. American Pit Bull Terrier
13. American Staffordshire Terrier
14. American Water Spaniel
15. Anatolian Shepherd Dog
16. Aussiedoodle
17. Australian Cattle Dog
18. Australian Kelpie
19. Australian Labradoodle
20. Australian Shepherd
21. Australian Silky Terrier
22. Australian Terrier
23. Australian Working Kelpie
24. Azawakh
25. Basenji
26. Bassador
27. Basset Bleu De Gascogne
28. Basset Fauve De Bretagne
29. Basset Hound
30. Bassugg
31. Bavarian Mountain Hound
32. Beagador
33. Beagle
34. Beaglier
35. Bearded Collie
36. Beauceron
37. Bedlington Terrier
38. Bedlington Whippet
39. Belgian Groenendael
40. Belgian Laekenois
41. Belgian Malinois
42. Belgian Shepherd
43. Belgian Tervuren
44. Bergamasco
45. Be

In [85]:
prompt = "What is the second dog breed beginning with the letter B"
url = "https://www.yappy.com/uk/dogs/breeds"
model = "gpt-4o"
response = fetch_webpage_and_query(prompt, url, model)
print(response)

The second dog breed beginning with the letter 'B' on the Yappy.com UK dog breed list is "Bassador."


all success

In [160]:
prompt = "What is the second dog breed beginning with the letter B"
url = "https://www.yappy.com/uk/dogs/breeds"
model = "gpt-3.5-turbo"
response = fetch_webpage_and_query(prompt, url, model)
print(response)

Error: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 61410 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}


fail

In [26]:
from scrapegraphai.graphs import SmartScraperGraph
import nest_asyncio
nest_asyncio.apply()


open_ai_key = "sk-YApzULCHGCaBlngoQKKjQqI9eMTjAlZsyBRIae3dMM8WvXXX"
graph_config = {
    "llm": {
        "api_key": open_ai_key,
        "model": "openai/gpt-4o",
        "base_url": "https://api.chatanywhere.tech/v1"
    },
    
    "depth": 1,  
    "only_inside_links": True,
    "verbose": True,
    "headless": True
    
    
}
graph = SmartScraperGraph(
    prompt="What is the second dog breed beginning with the letter B",
    source="https://www.yappy.com/uk/dogs/breeds",
    config=graph_config
)

result = graph.run()
print(result)

--- Executing Fetch Node ---
--- (Fetching HTML from: https://www.yappy.com/uk/dogs/breeds) ---
--- Executing ParseNode Node ---
--- Executing GenerateAnswer Node ---


{'second_dog_breed_beginning_with_B': 'Bassador'}


In [51]:
from scrapegraphai.graphs import CustomGraph
import nest_asyncio
nest_asyncio.apply()


open_ai_key = "sk-YApzULCHGCaBlngoQKKjQqI9eMTjAlZsyBRIae3dMM8WvXXX"
graph_config = {
    "llm": {
        "api_key": open_ai_key,
        "model": "openai/gpt-4o",
        "base_url": "https://api.chatanywhere.tech/v1"
    },
    
    "depth": 1,  
    "only_inside_links": True,
    "verbose": True,
    "headless": True
    
    
}
graph = CustomGraph(
    prompt="What is the lifespan of this dog",
    source="https://www.yappy.com/uk/bassador",
    config=graph_config
)

result = graph.run()
print(result)

--- Executing Fetch Node ---
--- (Fetching HTML from: https://www.yappy.com/uk/bassador) ---
--- Executing ParseNode Node ---
--- Executing GenerateAnswer Node ---


{'lifespan': '10 - 12 Years'}


In [30]:
from scrapegraphai.graphs import CustomGraph
import nest_asyncio
nest_asyncio.apply()


open_ai_key = "sk-YApzULCHGCaBlngoQKKjQqI9eMTjAlZsyBRIae3dMM8WvXXX"
graph_config = {
    "llm": {
        "api_key": open_ai_key,
        "model": "openai/gpt-3.5-turbo",
        "base_url": "https://api.chatanywhere.tech/v1"
    },
    
    "depth": 1,  
    "only_inside_links": True,
    "verbose": True,
    "headless": True
    
    
}
graph = CustomGraph(
    prompt="What is the lifespan of this dog",
    source="https://www.yappy.com/uk/bassador",
    config=graph_config
)

result = graph.run()
print(result)

--- Executing FetchLevelK Node ---
Fetching HTML content from: https://www.yappy.com/uk/bassador
Extracted 226 links.
--- Executing ParseNode Node ---
--- Executing RAG Node ---
--- Executing GenerateAnswer Node ---
Processing chunks: 100%|█████████████████████████████████████████████████████████████████████████| 2/2 [00:00<?, ?it/s]


{'Lifespan': '10 - 12 Years', 'Height': 'Males and Females 38cm - 46cm', 'Weight': 'Males and Females 22kg - 32kg', 'Coat': 'Short, Dense and Smooth', 'Color': 'Tan, Brown, Black, Fawn', 'Eye color': 'Brown', 'Common health issues': 'Hip Dysplasia, Gastric Dilatation-Volvulus (Bloat), IVDD', 'Other Names': 'Basset Lab, Bassetdor'}


gpt4o attempts 10 times，success 9 times ,1 get wrong answer.
gpt3.5 attempts 10 times， success 10 times

In [19]:
#with gpt3.5
URL="https://www.yappy.com/uk/bassador"
model="gpt-4o"
response = requests.get(URL)
response.raise_for_status()  # Ensure we raise an error for bad responses
html_content = response.text
import json
instruction = {
    "prompt": "What is the lifespan of this dog",
    "URL": URL,
    "html":html_content,
}
instruction_str = json.dumps(instruction)
response = get_scraper_task(instruction_str,model)
print(response)

Error: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 30130 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}


gpt3.5 ,fail

In [25]:
#with gpt4o
response = requests.get("https://www.yappy.com/uk/bassador")
response.raise_for_status()  # Ensure we raise an error for bad responses
html_content = response.text
import json
instruction = {
    "prompt": "What is the lifespan of this dog",
    "URL": "https://www.yappy.com/uk/bassador",
    "html":html_content,
}
instruction_str = json.dumps(instruction)
response = get_scraper_task(instruction_str)
print(response)

The lifespan of a Bassador is 10 - 12 years.


gpt4o attempts 10 times, success 10 times

In [70]:
from scrapegraphai.graphs import CustomGraph
import nest_asyncio
nest_asyncio.apply()


open_ai_key = "sk-YApzULCHGCaBlngoQKKjQqI9eMTjAlZsyBRIae3dMM8WvXXX"
graph_config = {
    "llm": {
        "api_key": open_ai_key,
        "model": "openai/gpt-4o",
        "base_url": "https://api.chatanywhere.tech/v1"
    },
    
    "depth": 1,  
    "only_inside_links": True,
    "verbose": True,
    "headless": True
    
    
}
graph = CustomGraph(
    prompt="List me all the dog breeds with their urls",
    source="https://www.yappy.com/uk/dogs/breeds",
    config=graph_config
)

result = graph.run()
print(result)

--- Executing FetchLevelK Node ---
Fetching HTML content from: https://www.yappy.com/uk/dogs/breeds
Extracted 535 links.
--- Executing ParseNode Node ---
--- Executing RAG Node ---
--- Executing GenerateAnswer Node ---
Processing chunks: 100%|███████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 3756.09it/s]


{'dog_breeds': {'Affenpinscher': None, 'Afghan Hound': None, 'Airedale Terrier': None, 'Akita': None, 'Alaskan Klee Kai': None, 'Alaskan Malamute': None, 'American Bulldog': None, 'American Cocker Spaniel': None, 'American Eskimo Dog': None, 'American Hairless Terrier': None, 'American Leopard Hound': None, 'American Pit Bull Terrier': None, 'American Staffordshire Terrier': None, 'American Water Spaniel': None, 'Anatolian Shepherd Dog': None, 'Aussiedoodle': None, 'Australian Cattle Dog': None, 'Australian Kelpie': None, 'Australian Labradoodle': None, 'Australian Shepherd': None, 'Australian Silky Terrier': None, 'Australian Terrier': None, 'Australian Working Kelpie': None, 'Azawakh': None, 'Basenji': None, 'Bassador': None, 'Basset Bleu De Gascogne': None, 'Basset Fauve De Bretagne': None, 'Basset Hound': None, 'Bassugg': None, 'Bavarian Mountain Hound': None, 'Beagador': None, 'Beagle': None, 'Beaglier': None, 'Bearded Collie': None, 'Beauceron': None, 'Bedlington Terrier': None, 

In [49]:
from scrapegraphai.graphs import CustomGraph
import nest_asyncio
nest_asyncio.apply()


open_ai_key = "sk-YApzULCHGCaBlngoQKKjQqI9eMTjAlZsyBRIae3dMM8WvXXX"
graph_config = {
    "llm": {
        "api_key": open_ai_key,
        "model": "openai/gpt-4o",
        "base_url": "https://api.chatanywhere.tech/v1"
    },
    
    "depth": 1,  
    "only_inside_links": True,
    "verbose": True,
    "headless": True
    
    
}
graph = CustomGraph(
    prompt="List me all the dog breeds with their urls",
    source="https://www.yappy.com/uk/dogs/breeds",
    config=graph_config
)

result = graph.run()
print(result)

--- Executing Fetch Node ---
--- (Fetching HTML from: https://www.yappy.com/uk/dogs/breeds) ---
--- Executing ParseNode Node ---
--- Executing GenerateAnswer Node ---


{'Affenpinscher': '/uk/affenpinscher', 'Afghan Hound': '/uk/afghanhound', 'Airedale Terrier': '/uk/airedaleterrier', 'Akita': '/uk/akita', 'Alaskan Klee Kai': '/uk/alaskankleekai', 'Alaskan Malamute': '/uk/alaskanmalamute', 'American Bulldog': '/uk/americanbulldog', 'American Cocker Spaniel': '/uk/americancockerspaniel', 'American Eskimo Dog': '/uk/americaneskimodog', 'American Hairless Terrier': '/uk/americanhairlessterrier', 'American Leopard Hound': '/uk/americanleopardhound', 'American Pit Bull Terrier': '/uk/americanpittbullterrier', 'American Staffordshire Terrier': '/uk/americanstaffordshireterrier', 'American Water Spaniel': '/uk/americanwaterspaniel', 'Anatolian Shepherd Dog': '/uk/anatolianshepherddog', 'Aussiedoodle': '/uk/aussiedoodle', 'Australian Cattle Dog': '/uk/australiancattledog', 'Australian Kelpie': '/uk/australiankelpie', 'Australian Labradoodle': '/uk/australianlabradoodle', 'Australian Shepherd': '/uk/australianshepherd', 'Australian Silky Terrier': '/uk/austral

2 times successfully geting task but miss some urls(wrong awser)
8 times success

In [43]:
#with gpt4o
response = requests.get("https://www.yappy.com/uk/dogs/breeds")
response.raise_for_status()  # Ensure we raise an error for bad responses
html_content = response.text
import json
instruction = {
    "prompt": "List me all the dog breeds with their urls(show all the results)",
    "URL": "https://www.yappy.com/uk/dogs/breeds",
    "html":html_content,
}
instruction_str = json.dumps(instruction)
response = get_scraper_task(instruction_str)
print(response)

Here is a list of dog breeds along with their URLs extracted from the Yappy website:

- **Affenpinscher**: [https://www.yappy.com/uk/affenpinscher](https://www.yappy.com/uk/affenpinscher)
- **Afghan Hound**: [https://www.yappy.com/uk/afghanhound](https://www.yappy.com/uk/afghanhound)
- **Airedale Terrier**: [https://www.yappy.com/uk/airedaleterrier](https://www.yappy.com/uk/airedaleterrier)
- **Akita**: [https://www.yappy.com/uk/akita](https://www.yappy.com/uk/akita)
- **Alaskan Klee Kai**: [https://www.yappy.com/uk/alaskankleekai](https://www.yappy.com/uk/alaskankleekai)
- **Alaskan Malamute**: [https://www.yappy.com/uk/alaskanmalamute](https://www.yappy.com/uk/alaskanmalamute)
- **American Bulldog**: [https://www.yappy.com/uk/americanbulldog](https://www.yappy.com/uk/americanbulldog)
- **American Cocker Spaniel**: [https://www.yappy.com/uk/americancockerspaniel](https://www.yappy.com/uk/americancockerspaniel)
- **American Eskimo Dog**: [https://www.yappy.com/uk/americaneskimodog](htt

10 time attempts,all can not show all the results due to token limits(wrong answer)

# newspage

In [90]:
from scrapegraphai.graphs import CustomGraph
import nest_asyncio
nest_asyncio.apply()


open_ai_key = "sk-YApzULCHGCaBlngoQKKjQqI9eMTjAlZsyBRIae3dMM8WvXXX"
graph_config = {
    "llm": {
        "api_key": open_ai_key,
        "model": "openai/gpt-4o",
        "base_url": "https://api.chatanywhere.tech/v1"
    },
    
    "depth": 1,  
    "only_inside_links": True,
    "verbose": True,
    "headless": True
    
    
}
graph = CustomGraph(
    prompt="List me all the titles",
    source="https://www.bbc.com/news",
    config=graph_config
)

result = graph.run()
print(result)

--- Executing Fetch Node ---
--- (Fetching HTML from: https://www.bbc.com/news) ---
--- Executing ParseNode Node ---
--- Executing GenerateAnswer Node ---


{'titles': ["China's crackdown on Hong Kong means protests are just a defiant memory", 'More Russian strikes as Syrian rebels advance further after taking Aleppo', 'Twelve dead, 135,000 displaced in worst Malaysia and Thailand floods in decades', 'Irish elections: Political parties plot path to government', "'I spent 30 years as a therapist to killers - and no-one is born evil'", "Who are the rebels seizing control of Syria's second city?", 'Woman searching for birth parents found dad was a friend on Facebook', 'Inside the ancient Indian ritual where humans become gods', 'UK MasterChef host hits out at accusers after allegations over sexualised comments', "'They threw her body into the ocean' - woman dies on boat headed for French island", 'Trump picks loyalist ex-aide as FBI director', "Belgium's sex workers get maternity leave and pensions under world-first law", "Georgia's pro-EU president vows to stay 'until someone is legitimately elected'", "'Our generation is lonelier so we're f

10 times attempt，9 times success,1 time miss something

In [87]:
prompt = "List me all the titles"
url = "https://www.bbc.com/news"
model = "gpt-4o"
response = fetch_webpage_and_query(prompt, url, model)
print(response)

Here are the titles from the BBC News homepage:

1. China's crackdown on Hong Kong means protests are just a defiant memory
2. More Russian strikes as Syrian rebels advance further after taking Aleppo
3. Who are the rebels seizing control of Syria's second city?
4. Woman searching for birth parents found dad was a friend on Facebook
5. Inside the ancient Indian ritual where humans become gods
6. Twelve dead, 135,000 displaced in worst Malaysia and Thailand floods in decades
7. Irish elections: Political parties plot path to government
8. 'I spent 30 years as a therapist to killers - and no-one is born evil'
9. UK MasterChef host hits out at accusers after allegations over sexualised comments
10. 'They threw her body into the ocean' - woman dies on boat headed for French island
11. Trump picks loyalist ex-aide as FBI director
12. Belgium's sex workers get maternity leave and pensions under world-first law
13. Georgia's pro-EU president vows to stay 'until someone is legitimately elected

10 times attempt,7 times success,2 times miss something, 1 time missuderstanding a link as a title.

In [9]:

from scrapegraphai.graphs import SmartScraperGraph
import nest_asyncio
nest_asyncio.apply()


open_ai_key = "sk-YApzULCHGCaBlngoQKKjQqI9eMTjAlZsyBRIae3dMM8WvXXX"
graph_config = {
    "llm": {
        "api_key": open_ai_key,
        "model": "openai/gpt-4o",
        "base_url": "https://api.chatanywhere.tech/v1"
    },
    
    "depth": 1,  
    "only_inside_links": True,
    "verbose": True,
    "headless": True
    
    
}
graph = SmartScraperGraph(
    prompt="List me all the titles",
    source="https://www.theguardian.com/uk",
    config=graph_config
)

result = graph.run()
print(result)

--- Executing Fetch Node ---
--- (Fetching HTML from: https://www.theguardian.com/uk) ---
--- Executing ParseNode Node ---
--- Executing GenerateAnswer Node ---


{'titles': ['Technology UK underestimates threat of cyber-attacks from hostile states and gangs, says security chief', 'Technology ‘Russia can turn the lights off’: how the UK is preparing for cyberwar', 'Housing Radical planning reform needed to hit 1.5m new homes target in England, Labour warned', 'Business Jaguar boss says it has shown ‘fearless creativity’ with new electric car', 'UK South Wales residents displaced by sinkhole fear they may not return by Christmas', 'Television BBC rejects call to cut MasterChef episodes as Gregg Wallace apologises', 'Foreign policy Starmer rejects false choice between Trump’s US and EU in key speech', 'World Vietnamese tycoon faces scramble to raise billions to avoid death sentence', 'Communities UK towns that saw rioting last summer have ‘torn social fabric’, report claims', 'Medicine Pioneering drug for rare eye cancer gets go-ahead to be offered on NHS', 'Society Over-60s who live apart from partners have better wellbeing, study finds', 'Analys

In [20]:
prompt = "List me all the titles"
url = "https://www.theguardian.com/uk"
model = "gpt-4o"
response = fetch_webpage_and_query(prompt, url, model)
print(response)

Error: Error code: 400 - {'error': {'message': "Invalid 'messages[1].content': string too long. Expected a string with maximum length 1048576, but got a string with length 1202198 instead.", 'type': 'invalid_request_error', 'param': 'messages[1].content', 'code': 'string_above_max_length'}}


In [5]:
from scrapegraphai.graphs import CustomGraph
import nest_asyncio
nest_asyncio.apply()


open_ai_key = "sk-YApzULCHGCaBlngoQKKjQqI9eMTjAlZsyBRIae3dMM8WvXXX"
graph_config = {
    "llm": {
        "api_key": open_ai_key,
        "model": "openai/gpt-4o",
        "base_url": "https://api.chatanywhere.tech/v1"
    },
    
    "depth": 1,  
    "only_inside_links": True,
    "verbose": True,
    "headless": True
    
    
}
graph = CustomGraph(
    prompt="what is the second most watched video in this page",
    source="https://www.bbc.com/news",
    config=graph_config
)

result = graph.run()
print(result)

--- Executing FetchLevelK Node ---
Fetching HTML content from: https://www.bbc.com/news
Fetched content for https://www.bbc.com/news at depth 1
Extracted 190 links.
--- Executing ParseNode Node ---
--- Executing RAG Node ---
Creating collection 'vectorial_collection'...
Upserted 2 documents into the vector database.
--- Executing GenerateAnswer Node ---
Processing chunks: 100%|█████████████████████████████████████████████████████████████████████████| 2/2 [00:00<?, ?it/s]


{'second_most_watched_video': {'title': 'Fireworks fired at police during Georgia protests', 'link': '/news/videos/crmnw3npke0o'}}


attempt 10 times , 8 times success, 2 times can not find the video(Return a NA)

In [150]:
from scrapegraphai.graphs import CustomGraph
import nest_asyncio
nest_asyncio.apply()


open_ai_key = "sk-YApzULCHGCaBlngoQKKjQqI9eMTjAlZsyBRIae3dMM8WvXXX"
graph_config = {
    "llm": {
        "api_key": open_ai_key,
        "model": "openai/gpt-3.5-turbo",
        "base_url": "https://api.chatanywhere.tech/v1"
    },
    
    "depth": 1,  
    "only_inside_links": True,
    "verbose": True,
    "headless": True
    
    
}
graph = CustomGraph(
    prompt="what is the second most watched video in this page",
    source="https://www.bbc.com/news",
    config=graph_config
)

result = graph.run()
print(result)

--- Executing Fetch Node ---
--- (Fetching HTML from: https://www.bbc.com/news) ---
--- Executing ParseNode Node ---
--- Executing GenerateAnswer Node ---


{'answer': "You can run but you can't hide' - fugitive expert"}


In [152]:
from scrapegraphai.graphs import CustomGraph
import nest_asyncio
nest_asyncio.apply()


open_ai_key = "sk-YApzULCHGCaBlngoQKKjQqI9eMTjAlZsyBRIae3dMM8WvXXX"
graph_config = {
    "llm": {
        "api_key": open_ai_key,
        "model": "openai/gpt-3.5-turbo",
        "base_url": "https://api.chatanywhere.tech/v1"
    },
    
    "depth": 1,  
    "only_inside_links": True,
    "verbose": True,
    "headless": True
    
    
}
graph = CustomGraph(
    prompt="what is the second most watched video in this page",
    source="https://www.bbc.com/news",
    config=graph_config
)

result = graph.run()
print(result)

--- Executing FetchLevelK Node ---
Fetching HTML content from: https://www.bbc.com/news
Extracted 187 links.
--- Executing ParseNode Node ---
--- Executing RAG Node ---
--- Executing GenerateAnswer Node ---
Processing chunks: 100%|█████████████████████████████████████████████████████████████████████████| 2/2 [00:00<?, ?it/s]


{'answer': 'Video', 'question': 'What section of the website can you find videos?', 'options': ['/innovation', '/culture', '/arts', '/travel', '/future-planet', '/video', '/live', 'https://www.bbc.co.uk/sounds', 'https://www.bbc.com/weather', 'https://shop.bbc.com/']}



10 times attempt，5 success,3 wrong answer，2 times fail

In [122]:
prompt = "what is the second most watched video in this page"
url = "https://www.bbc.com/news"
model = "gpt-4o"
response = fetch_webpage_and_query(prompt, url, model)
print(response)

The second most watched video on the BBC News page is titled **"'You can run but you can't hide' - fugitive expert"**. You can watch it [here](https://www.bbc.com/news/videos/cz9x9wqwz7lo).


attempt 10 times , 9 times success

In [110]:
prompt = "what is the second most watched video in this page"
url = "https://www.bbc.com/news"
model = "gpt-3.5-turbo"
response = fetch_webpage_and_query(prompt, url, model)
print(response)

Error: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 124179 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}


gpt3.5 failed